In [2]:
!nvidia-smi

zsh:1: command not found: nvidia-smi


In [2]:
from google.colab import files
uploaded = files.upload()

Saving dev.csv to dev.csv
Saving test.csv to test.csv
Saving train.csv to train.csv


In [3]:
import numpy as np
import torch
from torch import nn

def row_solver(row):
    rowlist = row.split(',')
    rowid , label = row[0], row[-1]
    sent = ' '.join(rowlist[1:-1])
    return rowid , sent, label

def load_data(fn = 'ntu-adl-hw0-spring-2021/train.csv'):

    with open(fn , 'r') as f:
        data = f.read()
    data = [row_solver(row) for row in data.split('\n') if row.replace(' ','') != ''][1:]
    x,y = [row[1].split() for row in data] , [int(row[2]) for row in data]
    return x,y


x,y = load_data()


In [6]:
print(x[:3])
print(y[:3])

[['什麼', '才', '是', '想要', '的'], ['應該', '是', '學姊', '吧', '！'], ['我', '想要', '瘦', '啊', '希望', '能', '趁', '這', '次', '暑假', '來', '個', '減肥', '大計', '祈禱', '我', '能', '持之以恆', '啊', '看看', '能不能', '再', '開學', '後', '來', '個', '好', '桃花', 'Sent', 'from', 'my', 'Android']]
[0, 1, 1]


In [7]:
!ls

adlWorkspace.ipynb          ntu-adl-hw0-spring-2021.zip
ntu-adl-hw0-spring-2021


In [8]:

class Tokenizer:
    def __init__(self , sents):
        word_set = set()
        word_idx = {'':0}
        
        for sent in sents:
            # print(sent)
            word_set |=set(sent)
        for i, word in enumerate(list(word_set)):
            word_idx[word]=i+1

        self.word_set = word_set
        self.word_idx = word_idx
        self.num_word = len(word_set)

        
    def BOW(self,  sents):
        output = np.zeros((len(sents), self.num_word+1 )) # OTHER: 0

        for row_id,sent in enumerate(sents):

            ids = [self.word_idx.get(word,0) for word in sent]
            for idx in ids:
                output[row_id,idx]+=1

        return output





In [9]:
tokenizer = Tokenizer(x)

In [10]:
x_train = tokenizer.BOW(x)
y_train = np.array(y)
x_train.shape


(80000, 75653)

In [11]:
# from torch import nn
# import torch
import tensorflow as tf


In [12]:
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(tokenizer.num_word+1, 128,
                              input_shape=(None,tokenizer.num_word+1)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(1, activation='sigmoid')
])

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, None, 75653, 128)  9683584   
_________________________________________________________________
dense (Dense)                (None, None, 75653, 128)  16512     
_________________________________________________________________
dense_1 (Dense)              (None, None, 75653, 1)    129       
Total params: 9,700,225
Trainable params: 9,700,225
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(x_train,y_train,
         epochs=100,
         batch_size= 128,
         verbose=2)

Epoch 1/100


In [65]:
x_train[:3].sum(axis=1)

array([ 5.,  5., 32.])

In [67]:
x_train.dtype

dtype('float64')